In [11]:
import reservoirpy as rpy
import pandas as pd
import numpy as np

# rpy.verbosity(0)  # no need to be too verbose here
rpy.set_seed(42)  # make everything reproducible !

In [12]:
from reservoirpy.nodes import Reservoir
# Create the reservoir
reservoir = Reservoir(500, lr=0.5, sr=0.9)

In [13]:
# Load the data
data = pd.read_csv('incidents_and_temperatures_per_day.csv', header=0)
# Drop nan values if present
data = data.dropna() if data.isnull().values.any() else data
# Retrieve the needed data columns
X = data[['temperature', 'humidite', 'nuages']]
Y = data[['seuil']]

In [14]:
# Prepare the data for the reservoir (needs to be a numpy array)
X = np.array(X)
Y = np.array(Y)

print(np.unique(Y))

# Transform Y into one-hot encoding with numpy
Y = np.eye(3)[Y].reshape(-1, 3)
print(Y.shape)

[0 1 2]
(3286, 3)


In [15]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
lx, ly = len(X), len(Y)
X_train, X_test, Y_train, Y_test = X[:int(lx*0.8)], X[int(lx*0.8):], Y[:int(ly*0.8)], Y[int(ly*0.8):]

In [16]:
from reservoirpy.nodes import Ridge
ridge = Ridge(ridge=1e-7)
esn_model = reservoir >> ridge

In [17]:
train_states = reservoir.run(X_train)

esn_model = esn_model.fit(X_train, Y_train, warmup=10)

Running Reservoir-1: 100%|██████████| 2628/2628 [00:00<00:00, 5828.97it/s]
Running Model-1: 2628it [00:00, 5087.14it/s]<?, ?it/s]
Running Model-1: 100%|██████████| 1/1 [00:00<00:00,  1.81it/s]


Fitting node Ridge-1...


In [18]:
# Check if the method has initialized all nodes in the Model, and trained the Ridge readout
print(reservoir.is_initialized, ridge.is_initialized, ridge.fitted)

True True True


In [19]:
# Predict 
Y_pred = esn_model.run(X_test)
Y_pred

Running Model-1: 658it [00:00, 4111.99it/s]           


array([[0.78170595, 0.1213185 , 0.09697531],
       [0.80088627, 0.18989583, 0.00921803],
       [0.74596224, 0.15919341, 0.09484454],
       ...,
       [0.64696162, 0.23095666, 0.12208189],
       [0.63428868, 0.27647743, 0.08923401],
       [0.62333907, 0.32826862, 0.04839244]])

In [20]:
# compute accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1)))

# compute confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test.argmax(axis=1), Y_pred.argmax(axis=1)))

0.48632218844984804
[[282  29  16]
 [167  18  27]
 [ 87  12  20]]
